# Azure OpenAI Assistants

Azure OpenAI Assistants allows you to create AI assistants tailored to your needs through custom instructions and augmented by advanced tools like code interpreter, and custom functions. In this article we'll provide an in-depth walkthrough of getting started with the Assistants API.

In [13]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2024-05-01-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

# Create an assistant
assistant = client.beta.assistants.create(
    name="Data Visualization",
    instructions=f"You are a helpful AI assistant who makes interesting visualizations based on data." 
    f"You have access to a sandboxed environment for writing and testing code."
    f"When you are asked to create a visualization you should follow these steps:"
    f"1. Write the code."
    f"2. Anytime you write new code display a preview of the code to show your work."
    f"3. Run the code to confirm that it runs."
    f"4. If the code is successful display the visualization."
    f"5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4o-mini" # Not all models are supported for Assitants, you can check the list here /concepts/models?tabs=python-secure%2Cglobal-standard%2Cstandard-chat-completions#assistants-preview
)

In [14]:
print(assistant.model_dump_json(indent=2))

{
  "id": "asst_q5teE79T7j2hGCc4EFkF9IzO",
  "created_at": 1734535859,
  "description": null,
  "instructions": "You are a helpful AI assistant who makes interesting visualizations based on data.You have access to a sandboxed environment for writing and testing code.When you are asked to create a visualization you should follow these steps:1. Write the code.2. Anytime you write new code display a preview of the code to show your work.3. Run the code to confirm that it runs.4. If the code is successful display the visualization.5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
  "metadata": {},
  "model": "gpt-4o-mini",
  "name": "Data Visualization",
  "object": "assistant",
  "tools": [
    {
      "type": "code_interpreter"
    }
  ],
  "response_format": "auto",
  "temperature": 1.0,
  "tool_resources": {
    "code_interpreter": {
      "file_ids": []
    },
    "file_search": null
  },
  "top_p"

In [15]:
# Create a thread
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_QILvMeP2NWlLDeF07zB67rks', created_at=1734535860, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [16]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Create a visualization of a sinewave"
)

In [17]:
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_y1dOe40xAfxyPPe54ZsdTUpV",
      "assistant_id": null,
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Create a visualization of a sinewave"
          },
          "type": "text"
        }
      ],
      "created_at": 1734535861,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "status": null,
      "thread_id": "thread_QILvMeP2NWlLDeF07zB67rks"
    }
  ],
  "object": "list",
  "first_id": "msg_y1dOe40xAfxyPPe54ZsdTUpV",
  "last_id": "msg_y1dOe40xAfxyPPe54ZsdTUpV",
  "has_more": false
}


We could also pass an instructions parameter here, but this would override the existing instructions that we have already provided for the assistant.

In [18]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

In [19]:
# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status
print(status)

completed


In [20]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_7rtShxsTg6w8ajRaxS8wFK0S",
      "assistant_id": "asst_q5teE79T7j2hGCc4EFkF9IzO",
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "image_file": {
            "file_id": "assistant-OqkRNoQaBGOfA98x3ccKHXAB",
            "detail": null
          },
          "type": "image_file"
        },
        {
          "text": {
            "annotations": [],
            "value": "Here is the visualization of a sine wave. The graph shows the sine function plotted against values from \\(0\\) to \\(2\\pi\\) radians. If you have any other requests or need further modifications, feel free to ask!"
          },
          "type": "text"
        }
      ],
      "created_at": 1734535869,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_Kfhaj3U19cH6aaZG7WMEq79I",
      "status": null,
      "thread_id": "threa

In [21]:
data = json.loads(messages.model_dump_json(indent=2))  # Load JSON data into a Python object
image_file_id = data['data'][0]['content'][0]['image_file']['file_id']

print(data)  

{'data': [{'id': 'msg_7rtShxsTg6w8ajRaxS8wFK0S', 'assistant_id': 'asst_q5teE79T7j2hGCc4EFkF9IzO', 'attachments': [], 'completed_at': None, 'content': [{'image_file': {'file_id': 'assistant-OqkRNoQaBGOfA98x3ccKHXAB', 'detail': None}, 'type': 'image_file'}, {'text': {'annotations': [], 'value': 'Here is the visualization of a sine wave. The graph shows the sine function plotted against values from \\(0\\) to \\(2\\pi\\) radians. If you have any other requests or need further modifications, feel free to ask!'}, 'type': 'text'}], 'created_at': 1734535869, 'incomplete_at': None, 'incomplete_details': None, 'metadata': {}, 'object': 'thread.message', 'role': 'assistant', 'run_id': 'run_Kfhaj3U19cH6aaZG7WMEq79I', 'status': None, 'thread_id': 'thread_QILvMeP2NWlLDeF07zB67rks'}, {'id': 'msg_y1dOe40xAfxyPPe54ZsdTUpV', 'assistant_id': None, 'attachments': [], 'completed_at': None, 'content': [{'text': {'annotations': [], 'value': 'Create a visualization of a sinewave'}, 'type': 'text'}], 'created

In [22]:
content = client.files.content(image_file_id)

image= content.write_to_file("sinewave.png")

In [23]:
from PIL import Image

# Display the image in the default image viewer
image = Image.open("sinewave.png")
image.show()

In [24]:
# Add a new user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Show me the code you used to generate the sinewave"
)

In [25]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions  but these will override the default instructions
)


In [26]:
# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
status = run.status
print(status)

completed


* Please wait until you get "completed"

In [27]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_D0u0LEhj1QfBuO9WDjwzMx2k",
      "assistant_id": "asst_q5teE79T7j2hGCc4EFkF9IzO",
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Here is the code I used to generate the sine wave visualization:\n\n```python\nimport numpy as np\nimport matplotlib.pyplot as plt\n\n# Generate data for the sine wave\nx = np.linspace(0, 2 * np.pi, 100)\ny = np.sin(x)\n\n# Create the plot\nplt.figure(figsize=(10, 5))\nplt.plot(x, y, label='Sine Wave', color='b')\nplt.title('Sine Wave Visualization')\nplt.xlabel('X values (radians)')\nplt.ylabel('Sine of X')\nplt.axhline(0, color='black', lw=0.5, ls='--')\nplt.axvline(0, color='black', lw=0.5, ls='--')\nplt.grid()\nplt.legend()\nplt.show()\n```\n\nLet me know if you need any modifications or additional visualizations!"
          },
          "type": "text"
        }
      ],
      "created_at": 1734535965,
      "inc

In [28]:
data = json.loads(messages.model_dump_json(indent=2))
code = data['data'][0]['content'][0]['text']['value']
print(code)

Here is the code I used to generate the sine wave visualization:

```python
import numpy as np
import matplotlib.pyplot as plt

# Generate data for the sine wave
x = np.linspace(0, 2 * np.pi, 100)
y = np.sin(x)

# Create the plot
plt.figure(figsize=(10, 5))
plt.plot(x, y, label='Sine Wave', color='b')
plt.title('Sine Wave Visualization')
plt.xlabel('X values (radians)')
plt.ylabel('Sine of X')
plt.axhline(0, color='black', lw=0.5, ls='--')
plt.axvline(0, color='black', lw=0.5, ls='--')
plt.grid()
plt.legend()
plt.show()
```

Let me know if you need any modifications or additional visualizations!


In [29]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I prefer visualizations in darkmode can you change the colors to make a darkmode version of this visualization."
)

# Run the thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [31]:
# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status
print(status)

completed


* Please wait until you get "completed"

In [32]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_n5crl3tXkSidhIcwSUwDzCWt",
      "assistant_id": "asst_q5teE79T7j2hGCc4EFkF9IzO",
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "image_file": {
            "file_id": "assistant-QsABw8YRRI9fWZY4gXWPSq1S",
            "detail": null
          },
          "type": "image_file"
        },
        {
          "text": {
            "annotations": [],
            "value": "Here is the dark mode version of the sine wave visualization. The colors have been adjusted to enhance visibility against a dark background. If you have any further requests or adjustments, let me know!"
          },
          "type": "text"
        }
      ],
      "created_at": 1734535995,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_y59vKitO7l9Bg18uPLrO3nxe",
      "status": null,
      "thread_id": "thread_QILvMeP2NWl

In [33]:
data = json.loads(messages.model_dump_json(indent=2))  # Load JSON data into a Python object
image_file_id = data['data'][0]['content'][0]['image_file']['file_id'] # index numbers can vary if you have had a different conversation over the course of the thread.

print(image_file_id)

content = client.files.content(image_file_id)
image= content.write_to_file("dark_sine.png")

# Display the image in the default image viewer
image = Image.open("dark_sine.png")
image.show()

assistant-QsABw8YRRI9fWZY4gXWPSq1S


* You can delete the resource

In [34]:
client.beta.assistants.delete(assistant.id)

AssistantDeleted(id='asst_q5teE79T7j2hGCc4EFkF9IzO', deleted=True, object='assistant.deleted')